# Generate Unplanned Visits dataframe

In [1]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

main_dir = '/Users/kenlocey/Desktop/Rush/CMS_HospitalArchives/'

In [2]:
df = pd.read_pickle(main_dir + 'Unplanned_Visits/CombinedFiles_Unplanned_Visits/' + 'Facility.pkl')
print('df:', df.shape)

df.drop(labels=['Address', 'City', 'County Name', 'Facility Name', 
                'End Date', 'Start Date', 'Footnote',
                'Phone Number', 'State', 'ZIP Code',
                'Compared to National', 'Higher Estimate', 'Lower Estimate',
                   ], axis=1, inplace=True)


df = df.filter(items=['Facility ID',
                      'file_month',
                      'file_year',
                      'Measure ID',
                      'Measure Name',
                      'Denominator', 
                      'Higher Estimate',
                      'Lower Estimate',
                      'Number of Patients',
                      'Number of Patients Returned',
                      'Score',
                     ], axis=1)

df.head()

df: (1402233, 22)


,Facility ID,file_month,file_year,Measure ID,Measure Name,Denominator,Number of Patients,Number of Patients Returned,Score
0,010001,01,2023,EDAC_30_AMI,Hospital return days for heart attack patients,319,305,81,1.9
1,010001,01,2023,EDAC_30_HF,Hospital return days for heart failure patients,755,618,211,21.8
2,010001,01,2023,EDAC_30_PN,Hospital return days for pneumonia patients,436,412,99,-1.5
3,010001,01,2023,OP_32,Rate of unplanned hospital visits after colono...,254,Not Applicable,Not Applicable,14.1
4,010001,01,2023,OP_35_ADM,Rate of inpatient admissions for patients rece...,214,Not Applicable,Not Applicable,10.2


In [3]:
#print(df['Measure ID'].unique(), '\n')
#print(df['Measure Name'].unique(), '\n')

df['Measure'] = df['Measure ID'] + ' — ' + df['Measure Name']

measures = sorted(df['Measure'].unique())
for m in measures:
    print(m)

EDAC_30_AMI — Hospital return days for heart attack patients
EDAC_30_HF — Hospital return days for heart failure patients
EDAC_30_PN — Hospital return days for pneumonia patients
OP-32 — Rate of unplanned hospital visits after colonoscopy (per 1,000 colonoscopies)
OP_32 — Rate of unplanned hospital visits after colonoscopy (per 1,000 colonoscopies)
OP_35_ADM — Rate of inpatient admissions for patients receiving outpatient chemotherapy
OP_35_ED — Rate of emergency department (ED) visits for patients receiving outpatient chemotherapy
OP_36 — Ratio of unplanned hospital visits after hospital outpatient surgery
READM_30_AMI — Acute Myocardial Infarction (AMI) 30-Day Readmission Rate
READM_30_AMI — Rate of readmission for heart attack patients
READM_30_CABG — Rate of readmission for CABG
READM_30_CABG — Rate of readmission for CABG surgery patients
READM_30_COPD — Rate of readmission for chronic obstructive pulmonary disease (COPD) patients
READM_30_HF — Heart failure (HF) 30-Day Readmissio

In [4]:
d = {
    'EDAC_30_AMI — Hospital return days for heart attack patients': 'EDAC-30 AMI — Hospital return days for AMI patients', 
    'EDAC_30_HF — Hospital return days for heart failure patients': 'EDAC-30 HF — Hospital return days for HF patients',
    'EDAC_30_PN — Hospital return days for pneumonia patients': 'EDAC-30 PN — Hospital return days for PN patients', 
    'OP-32 — Rate of unplanned hospital visits after colonoscopy (per 1,000 colonoscopies)': 'OP-32 — Rate of unplanned visits after colonoscopy (per 1K)', 
    'OP_32 — Rate of unplanned hospital visits after colonoscopy (per 1,000 colonoscopies)': 'OP-32 — Rate of unplanned visits after colonoscopy (per 1K)',
    
    'OP_35_ADM — Rate of inpatient admissions for patients receiving outpatient chemotherapy': 'OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo', 
    'OP_35_ED — Rate of emergency department (ED) visits for patients receiving outpatient chemotherapy': 'OP-35 ED — ED visit rate for patients receiving outpatient chemo', 
    'OP_36 — Ratio of unplanned hospital visits after hospital outpatient surgery': 'OP-36 — Ratio of unplanned visits after outpatient surgery', 
    'READM_30_AMI — Acute Myocardial Infarction (AMI) 30-Day Readmission Rate': 'READM-30 AMI — AMI 30-Day Readmission Rate', 
    'READM_30_AMI — Rate of readmission for heart attack patients': 'READM-30 AMI — AMI 30-Day Readmission Rate',
    'READM_30_CABG — Rate of readmission for CABG': 'READM-30 CABG — CABG 30-Day Readmission Rate',
    'READM_30_CABG — Rate of readmission for CABG surgery patients': 'READM-30 CABG — CABG 30-Day Readmission Rate',
    
    'READM_30_COPD — Rate of readmission for chronic obstructive pulmonary disease (COPD) patients': 'READM-30 COPD — COPD 30-Day Readmission Rate', 
    
    'READM_30_HF — Heart failure (HF) 30-Day Readmission Rate': 'READM-30 HF — 30-Day HF Readmission Rate', 
    'READM_30_HF — Rate of readmission for heart failure patients': 'READM-30 HF — 30-Day HF Readmission Rate',
    
    'READM_30_HIP_KNEE — Rate of readmission after hip/knee replacement': 'READM-30 HIP-KNEE — 30-Day Readmission Rate after hip/knee replacement', 
    
    'READM_30_HOSP_WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate': 'READM-30 HOSP-WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate', 
    'READM_30_HOSP_WIDE — Rate of readmission after discharge from hospital (hospital-wide)': 'READM-30 HOSP-WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate',
    
    'READM_30_PN — Pneumonia (PN) 30-Day Readmission Rate': 'READM-30 PN — 30-Day Pneumonia Readmission Rate', 
    'READM_30_PN — Rate of readmission for pneumonia patients': 'READM-30 PN — 30-Day Pneumonia Readmission Rate',
    
    'READM_30_STK — Rate of readmission for stroke patients': 'READM-30 STK — 30-Day Readmission Rate for stroke patients',
}

df['Measure'].replace(to_replace=d, inplace=True)
print(df['Measure'].unique())

['EDAC-30 AMI — Hospital return days for AMI patients'
 'EDAC-30 HF — Hospital return days for HF patients'
 'EDAC-30 PN — Hospital return days for PN patients'
 'OP-32 — Rate of unplanned visits after colonoscopy (per 1K)'
 'OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo'
 'OP-35 ED — ED visit rate for patients receiving outpatient chemo'
 'OP-36 — Ratio of unplanned visits after outpatient surgery'
 'READM-30 AMI — AMI 30-Day Readmission Rate'
 'READM-30 CABG — CABG 30-Day Readmission Rate'
 'READM-30 COPD — COPD 30-Day Readmission Rate'
 'READM-30 HF — 30-Day HF Readmission Rate'
 'READM-30 HIP-KNEE — 30-Day Readmission Rate after hip/knee replacement'
 'READM-30 HOSP-WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate'
 'READM-30 PN — 30-Day Pneumonia Readmission Rate'
 'READM-30 STK — 30-Day Readmission Rate for stroke patients']


In [5]:
df.drop(labels=['Measure Name', 'Measure ID'], axis=1, inplace=True)
df.head()

,Facility ID,file_month,file_year,Denominator,Number of Patients,Number of Patients Returned,Score,Measure
0,010001,01,2023,319,305,81,1.9,EDAC-30 AMI — Hospital return days for AMI pat...
1,010001,01,2023,755,618,211,21.8,EDAC-30 HF — Hospital return days for HF patients
2,010001,01,2023,436,412,99,-1.5,EDAC-30 PN — Hospital return days for PN patients
3,010001,01,2023,254,Not Applicable,Not Applicable,14.1,OP-32 — Rate of unplanned visits after colonos...
4,010001,01,2023,214,Not Applicable,Not Applicable,10.2,OP-35 ADM — Inpatient admit rate for patients ...


In [6]:
Measures = sorted(df['Measure'].unique())
main_df = pd.DataFrame(columns=['Facility ID', 'file_month', 'file_year'])

for i, mi in enumerate(Measures):
    
    tdf = df[df['Measure'] == mi]
    measures = sorted(tdf['Measure'].unique())
    
    df2 = pd.DataFrame(columns=['Facility ID', 'file_month', 'file_year'])

    for j, m in enumerate(measures):
    
        tdf2 = tdf[tdf['Measure'] == m]
        for n in list(tdf2):
            if n == 'Measure' or n in ['Facility ID', 'file_month', 'file_year']:
                continue

            else:
                tdf2[n] = pd.to_numeric(tdf2[n], errors='coerce')
                tdf2.rename(columns={n: m + ' (' + n + ')'}, inplace=True)
        
        tdf2.drop(labels=['Measure'], axis=1, inplace=True)
        
        df2 = df2.merge(tdf2, on=['Facility ID', 'file_month', 
                                          'file_year'], how='outer')
    
    main_df = main_df.merge(df2, on=['Facility ID', 'file_month', 
                                          'file_year'], how='outer')



In [7]:
del df2
df = main_df.copy(deep=True)
del main_df

print(df.shape)
#df = df.loc[:, ~df.T.duplicated(keep='first')]
#df.dropna(how='all', axis=1, inplace=True)
#print(df.shape)
df.head()


(107280, 63)


,Facility ID,file_month,file_year,EDAC-30 AMI — Hospital return days for AMI patients (Denominator),EDAC-30 AMI — Hospital return days for AMI patients (Number of Patients),EDAC-30 AMI — Hospital return days for AMI patients (Number of Patients Returned),EDAC-30 AMI — Hospital return days for AMI patients (Score),EDAC-30 HF — Hospital return days for HF patients (Denominator),EDAC-30 HF — Hospital return days for HF patients (Number of Patients),EDAC-30 HF — Hospital return days for HF patients (Number of Patients Returned),EDAC-30 HF — Hospital return days for HF patients (Score),EDAC-30 PN — Hospital return days for PN patients (Denominator),EDAC-30 PN — Hospital return days for PN patients (Number of Patients),EDAC-30 PN — Hospital return days for PN patients (Number of Patients Returned),EDAC-30 PN — Hospital return days for PN patients (Score),OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Denominator),OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Number of Patients),OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Number of Patients Returned),OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Score),OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Denominator),OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Number of Patients),OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Number of Patients Returned),OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Score),OP-35 ED — ED visit rate for patients receiving outpatient chemo (Denominator),OP-35 ED — ED visit rate for patients receiving outpatient chemo (Number of Patients),OP-35 ED — ED visit rate for patients receiving outpatient chemo (Number of Patients Returned),OP-35 ED — ED visit rate for patients receiving outpatient chemo (Score),OP-36 — Ratio of unplanned visits after outpatient surgery (Denominator),OP-36 — Ratio of unplanned visits after outpatient surgery (Number of Patients),OP-36 — Ratio of unplanned visits after outpatient surgery (Number of Patients Returned),OP-36 — Ratio of unplanned visits after outpatient surgery (Score),READM-30 AMI — AMI 30-Day Readmission Rate (Denominator),READM-30 AMI — AMI 30-Day Readmission Rate (Number of Patients),READM-30 AMI — AMI 30-Day Readmission Rate (Number of Patients Returned),READM-30 AMI — AMI 30-Day Readmission Rate (Score),READM-30 CABG — CABG 30-Day Readmission Rate (Denominator),READM-30 CABG — CABG 30-Day Readmission Rate (Number of Patients),READM-30 CABG — CABG 30-Day Readmission Rate (Number of Patients Returned),READM-30 CABG — CABG 30-Day Readmission Rate (Score),READM-30 COPD — COPD 30-Day Readmission Rate (Denominator),READM-30 COPD — COPD 30-Day Readmission Rate (Number of Patients),READM-30 COPD — COPD 30-Day Readmission Rate (Number of Patients Returned),READM-30 COPD — COPD 30-Day Readmission Rate (Score),READM-30 HF — 30-Day HF Readmission Rate (Denominator),READM-30 HF — 30-Day HF Readmission Rate (Number of Patients),READM-30 HF — 30-Day HF Readmission Rate (Number of Patients Returned),READM-30 HF — 30-Day HF Readmission Rate (Score),READM-30 HIP-KNEE — 30-Day Readmission Rate after hip/knee replacement (Denominator),READM-30 HIP-KNEE — 30-Day Readmission Rate after hip/knee replacement (Number of Patients),READM-30 HIP-KNEE — 30-Day Readmission Rate after hip/knee replacement (Number of Patients Returned),READM-30 HIP-KNEE — 30-Day Readmission Rate after hip/knee replacement (Score),READM-30 HOSP-WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate (Denominator),READM-30 HOSP-WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate (Number of Patients),READM-30 HOSP-WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate (Number of Patients Returned),READM-30 HOSP-WIDE — 30-Day Hospital-Wide All-Cause Unplanned Readmission Rate (Score),READM-30 PN — 30-Day Pneumonia Readmission Rate (Denominator),READM-30 PN — 30-Day Pn

In [8]:
for i, n in enumerate(list(df)):
    if n in ['Facility ID', 'file_month', 'file_year']:
        continue
            
    df.rename(columns={n: 'Unplanned Visits: ' + n}, inplace=True)
    
print(df.shape)
df.head()

(107280, 63)


,Facility ID,file_month,file_year,Unplanned Visits: EDAC-30 AMI — Hospital return days for AMI patients (Denominator),Unplanned Visits: EDAC-30 AMI — Hospital return days for AMI patients (Number of Patients),Unplanned Visits: EDAC-30 AMI — Hospital return days for AMI patients (Number of Patients Returned),Unplanned Visits: EDAC-30 AMI — Hospital return days for AMI patients (Score),Unplanned Visits: EDAC-30 HF — Hospital return days for HF patients (Denominator),Unplanned Visits: EDAC-30 HF — Hospital return days for HF patients (Number of Patients),Unplanned Visits: EDAC-30 HF — Hospital return days for HF patients (Number of Patients Returned),Unplanned Visits: EDAC-30 HF — Hospital return days for HF patients (Score),Unplanned Visits: EDAC-30 PN — Hospital return days for PN patients (Denominator),Unplanned Visits: EDAC-30 PN — Hospital return days for PN patients (Number of Patients),Unplanned Visits: EDAC-30 PN — Hospital return days for PN patients (Number of Patients Returned),Unplanned Visits: EDAC-30 PN — Hospital return days for PN patients (Score),Unplanned Visits: OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Denominator),Unplanned Visits: OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Number of Patients),Unplanned Visits: OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Number of Patients Returned),Unplanned Visits: OP-32 — Rate of unplanned visits after colonoscopy (per 1K) (Score),Unplanned Visits: OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Denominator),Unplanned Visits: OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Number of Patients),Unplanned Visits: OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Number of Patients Returned),Unplanned Visits: OP-35 ADM — Inpatient admit rate for patients receiving outpatient chemo (Score),Unplanned Visits: OP-35 ED — ED visit rate for patients receiving outpatient chemo (Denominator),Unplanned Visits: OP-35 ED — ED visit rate for patients receiving outpatient chemo (Number of Patients),Unplanned Visits: OP-35 ED — ED visit rate for patients receiving outpatient chemo (Number of Patients Returned),Unplanned Visits: OP-35 ED — ED visit rate for patients receiving outpatient chemo (Score),Unplanned Visits: OP-36 — Ratio of unplanned visits after outpatient surgery (Denominator),Unplanned Visits: OP-36 — Ratio of unplanned visits after outpatient surgery (Number of Patients),Unplanned Visits: OP-36 — Ratio of unplanned visits after outpatient surgery (Number of Patients Returned),Unplanned Visits: OP-36 — Ratio of unplanned visits after outpatient surgery (Score),Unplanned Visits: READM-30 AMI — AMI 30-Day Readmission Rate (Denominator),Unplanned Visits: READM-30 AMI — AMI 30-Day Readmission Rate (Number of Patients),Unplanned Visits: READM-30 AMI — AMI 30-Day Readmission Rate (Number of Patients Returned),Unplanned Visits: READM-30 AMI — AMI 30-Day Readmission Rate (Score),Unplanned Visits: READM-30 CABG — CABG 30-Day Readmission Rate (Denominator),Unplanned Visits: READM-30 CABG — CABG 30-Day Readmission Rate (Number of Patients),Unplanned Visits: READM-30 CABG — CABG 30-Day Readmission Rate (Number of Patients Returned),Unplanned Visits: READM-30 CABG — CABG 30-Day Readmission Rate (Score),Unplanned Visits: READM-30 COPD — COPD 30-Day Readmission Rate (Denominator),Unplanned Visits: READM-30 COPD — COPD 30-Day Readmission Rate (Number of Patients),Unplanned Visits: READM-30 COPD — COPD 30-Day Readmission Rate (Number of Patients Returned),Unplanned Visits: READM-30 COPD — COPD 30-Day Readmission Rate (Score),Unplanned Visits: READM-30 HF — 30-Day HF Readmission Rate (Denominator),Unplanned Visits: READM-30 HF — 30-Day HF Readmission Rate (Number of Patients),Unplanned Visits: READM-30 HF — 30-Day HF Readmission Rate (Number of Patients Returned),Unplanned Visits: READM-30 HF — 30-Day HF Readmission Rate (Score),Unplanned Visits: READM-30 HIP-KNEE — 3

In [9]:
df.to_pickle('dataframe_data/unplanned_visits_df.pkl', protocol=5)